## Álgebra Linear Computacional
### Fatorações básicas (Eliminação de Gauss, Pivoteamento Parcial e Cholesky)
#### Profs. Alexandre Salles da Cunha e Ana Paula Couto da Silva

In [2]:
import numpy as np
import math


#### Para explorar o conteúdo da biblioteca np (ou qualquer outra), digite np. e complete com "tab" para que o sistema ofereça as opções

In [3]:
A = np.array([[21,21,-7],[11,6,0],[-7,6,8]],dtype = 'float64')
R = np.random.random((4,4))
I = np.eye((4))
b = np.array([2,3,3,9],dtype='float64')
x = np.linalg.solve(R,b)
n = len(A)


In [7]:
def EliminacaoGauss(A):
    n = len(A)
    #U = A (esta instrucao difere da abaixo .copy(A))
    U = np.copy(A)
    L = np.eye(n)
    for j in range(0,n-1):
        for i in range(j+1,n):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:n] = U[i,j:n] - L[i,j]*U[j,j:n]
    return U,L

In [8]:
U,L = EliminacaoGauss(A)

/tmp/ipykernel_1335/402579995.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  L[i,j] = U[i,j]/U[j,j]
/tmp/ipykernel_1335/402579995.py:9: RuntimeWarning: invalid value encountered in multiply
  U[i,j:n] = U[i,j:n] - L[i,j]*U[j,j:n]
/tmp/ipykernel_1335/402579995.py:8: RuntimeWarning: invalid value encountered in double_scalars
  L[i,j] = U[i,j]/U[j,j]


In [9]:
print(U)
print(L)

[[  0.  -4. -16.]
 [ nan -inf -inf]
 [ nan  nan  nan]]
[[  1.   0.   0.]
 [-inf   1.   0.]
 [ inf  nan   1.]]


In [10]:
def EliminacaoGauss_Outer(A):
    # nao implementa troca de linhas
    n = len(A)
    U = np.full((n,n),0.0)
    L = np.eye(n)
    AC = np.copy(A)
    for j in range(0,n-1):
        L[j+1:,j] = AC[j+1:,j]/AC[j,j]
        U[j,j:]    = AC[j,j:]
        AC[j+1:,j+1:] = AC[j+1:,j+1:] - np.outer(L[j+1:,j],U[j,j+1:])
        #print(AC[j+1:,j+1:])
        #print(U)
    U[n-1,n-1] = AC[n-1,n-1]
    #print(L)
    #print(U)
    return L,U

In [11]:
L,U = EliminacaoGauss_Outer(A)
print(np.matmul(L,U)-A)

[[ 0. nan nan]
 [nan nan nan]
 [nan nan nan]]


/tmp/ipykernel_1335/1194319198.py:8: RuntimeWarning: divide by zero encountered in divide
  L[j+1:,j] = AC[j+1:,j]/AC[j,j]
/tmp/ipykernel_1335/1194319198.py:8: RuntimeWarning: invalid value encountered in divide
  L[j+1:,j] = AC[j+1:,j]/AC[j,j]
/tmp/ipykernel_1335/1352835577.py:2: RuntimeWarning: invalid value encountered in matmul
  print(np.matmul(L,U)-A)


In [5]:
def DeterminaLinhaPivot(U,j):
    vmax = abs(U[j,j])
    jmax = j
    for i in range(j+1,len(U)):
        if abs(U[i,j]) > vmax:
            vmax = abs(U[i,j])
            jmax = i
    return jmax

def Troca(U,L,j,p,pivot):
    for i in range(0,j):
        # parte em L
        t = L[j,i]
        L[j,i] = L[p,i]
        L[p,i] = t
    for i in range(j,len(U)):
        # parte em U
        t = U[j,i]
        U[j,i] = U[p,i]
        U[p,i] = t
    
    t = pivot[j]
    pivot[j] = pivot[p]
    pivot[p] = t
    

def EliminacaoGaussPivoteamento(A):
    n = len(A)
    U = np.copy(A)
    L = np.eye(n)
    pivot = np.arange(0,n,1)
    for j in range(0,n-1):
        p = DeterminaLinhaPivot(U,j)
        #print('pivot',p)
        if (p != j):
            Troca(U,L,j,p,pivot)
        for i in range(j+1,n):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:n] = U[i,j:n] - L[i,j]*U[j,j:n]
    return L,U,pivot

In [6]:
L,U,pivot = EliminacaoGaussPivoteamento(A)
print(L)
print(U)
print(pivot)

[[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.52380952 -0.38461538  1.        ]]
[[21.         21.         -7.        ]
 [ 0.         13.          5.66666667]
 [ 0.          0.          5.84615385]]
[0 2 1]


In [8]:
def EliminacaoGauss_Outer_Pivotal(A):
    # implementa pivoteamento parcial na visao por colunas
    n = len(A)
    pivot = np.arange(0,n,1)
    U = np.full((n,n),0.0)
    L = np.eye(n)
    AC = np.copy(A)
    for j in range(0,n-1):
        
        p = DeterminaLinhaPivot(AC,j)
        if (p != j):
            Troca(AC,L,j,p,pivot)
                
        L[j+1:,j] = AC[j+1:,j]/AC[j,j]
        U[j,j:]    = AC[j,j:]
        AC[j+1:,j+1:] = AC[j+1:,j+1:] - np.outer(L[j+1:,j],U[j,j+1:])
        #print(AC[j+1:,j+1:])
        #print(U)
    U[n-1,n-1] = AC[n-1,n-1]
    #print(L)
    #print(U)
    return L,U,pivot

In [9]:
L,U,pivot = EliminacaoGauss_Outer_Pivotal(A)
print(L)
print(U)
print(pivot)

[[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.52380952 -0.38461538  1.        ]]
[[21.         21.         -7.        ]
 [ 0.         13.          5.66666667]
 [ 0.          0.          5.84615385]]
[0 2 1]


In [16]:
import scipy
import scipy.linalg
perm,l,u = scipy.linalg.lu(A)
print(l)
print(u)
print(perm)

[[ 1.          0.          0.        ]
 [-0.5         1.          0.        ]
 [ 0.          0.88888889  1.        ]]
[[ 28.         -39.         -12.        ]
 [  0.          -4.5          6.        ]
 [  0.           0.         -21.33333333]]
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [17]:
def ResolveTriangInf(L,b,p):
    n = len(L)
    y = np.zeros(n,dtype = 'float64')
    for i in range(0,n):
        y[i] = b[p[i]] - np.matmul(L[i,:i],y[:i])
        if L[i,i] != 0.0:
            y[i] = y[i]/L[i,i]
    return y

def ResolveTriangSup(U,b):
    n = len(U)
    x = np.zeros(n,dtype = 'float64')
    for i in range(n-1,-1,-1):
        x[i] = b[i] - np.matmul(U[i,i+1:],x[i+1:])
        if (U[i,i] != 0.0):
            x[i] = x[i] / U[i,i]
    return x    

def ResolveSistemaLinear(A,b):
    L,U,pivot = EliminacaoGaussPivoteamento(A)
    y = ResolveTriangInf(L,b,pivot)
    x = ResolveTriangSup(U,y)
    return x

In [18]:
y = ResolveTriangInf(L,b,pivot)
print(b)
print(pivot)
print(y)

[2. 3. 3. 9.]
[2 1 0]
[ 3.   4.5 -2. ]


In [19]:
Y = range(len(L)-1,-1,-1)
for i in Y:
    print(i)

2
1
0


In [20]:
x = ResolveSistemaLinear(A,b)
print(x)

[-1.07142857 -0.875       0.09375   ]


In [21]:
ATA = A @ A.transpose()
print(ATA)

[[  272.  -252.   348.]
 [ -252.   565. -1121.]
 [  348. -1121.  2449.]]


In [22]:
def Cholesky_Outer(R):
    n = len(R)
    for j in range(0,n):
        if (R[j,j] > 0.0):
            R[j,j] = math.sqrt(R[j,j])
            R[j,j+1:] = R[j,j+1:]/R[j,j]
            R[j+1:,j+1:] = R[j+1:,j+1:] - np.outer(R[j,j+1:],R[j,j+1:])
    return np.triu(R)        
            
        

In [23]:
R = ATA.copy()
Fator = Cholesky_Outer(R)
print(Fator)

[[ 16.4924225  -15.27974438  21.10059938]
 [  0.          18.20794914 -43.85931822]
 [  0.           0.           8.95125194]]


In [24]:
Fator2 = np.linalg.cholesky(ATA)
print(Fator2)
AR = np.matmul(Fator2,Fator)
print(AR)

[[ 16.4924225    0.           0.        ]
 [-15.27974438  18.20794914   0.        ]
 [ 21.10059938 -43.85931822   8.95125194]]
[[  272.  -252.   348.]
 [ -252.   565. -1121.]
 [  348. -1121.  2449.]]
